# 🍎 Phi-4 Model with AIProjectClient 🍏

**Phi-4** is a next-generation open model that aims to provide near GPT-4o capabilities at a fraction of the cost, making it ideal for many enterprise or personal use cases. It's especially great for chain-of-thought reasoning and RAG (Retrieval Augmented Generation) scenarios.

In this notebook, you'll see how to:
1. **Initialize** an `AIProjectClient` for your Azure AI Foundry environment.
2. **Chat** with the **Phi-4** model using `azure-ai-inference`.
3. **Show** a Health & Fitness example, featuring disclaimers and wellness Q&A.
4. **Enjoy** the value proposition of a cheaper alternative to GPT-4 with strong reasoning capabilities. 🏋️

> **Disclaimer**: This is not medical advice. Please consult professionals.

## Why Phi-4?
Phi-4 is a 14B-parameter model trained on curated data for high reasoning performance.
- **Cost-Effective**: Get GPT-4-level performance for many tasks without the GPT-4 price.
- **Reasoning & RAG**: Perfect for chain-of-thought reasoning steps and retrieval augmented generation workflows.
- **Generous Context Window**: 16K tokens, enabling more context or longer user conversations.

<img src="./seq-diagrams/4-phi-4.png" width="30%"/>


## 1. Setup

Below, we'll install and import the necessary libraries:
- **azure-ai-projects**: For the `AIProjectClient`.
- **azure-ai-inference**: For calling your model, specifically the chat completions.
- **azure-identity**: For `DefaultAzureCredential`.

Ensure you have a `.env` file with:
```bash
PROJECT_CONNECTION_STRING=<your-conn-string>
SERVERLESS_MODEL_NAME=phi-4
```

> **Note**: It's recommended to complete the [`3-basic-rag.ipynb`](./3-basic-rag.ipynb) notebook before this one, as it covers important concepts that will be helpful here.

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage

from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent.parent
load_dotenv(parent_dir / '.env')

conn_string = os.getenv("PROJECT_CONNECTION_STRING")
phi4_deployment = os.getenv("SERVERLESS_MODEL_NAME", "phi-4")

try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=conn_string,
    )
    print("✅ AIProjectClient created successfully!")
except Exception as e:
    print("❌ Error creating AIProjectClient:", e)

✅ AIProjectClient created successfully!


## 2. Chat with Phi-4 🍏
We'll demonstrate a simple conversation using **Phi-4** in a health & fitness context. We'll define a system prompt that clarifies the role of the assistant. Then we'll ask some user queries.

> Notice that Phi-4 is well-suited for chain-of-thought reasoning. We'll let it illustrate its reasoning steps for fun.


In [2]:
def chat_with_phi4(user_question, chain_of_thought=False):
    """Send a chat request to the Phi-4 model with optional chain-of-thought."""
    # We'll define a system message with disclaimers:
    system_prompt = (
        "You are a Phi-4 AI assistant, focusing on health and fitness.\n"
        "Remind users that you are not a medical professional, but can provide general info.\n"
    )

    # We can optionally instruct the model to show chain-of-thought. (Use carefully in production.)
    if chain_of_thought:
        system_prompt += "Please show your step-by-step reasoning in your answer.\n"

    # We create messages for system + user.
    system_msg = SystemMessage(content=system_prompt)
    user_msg = UserMessage(content=user_question)

    with project_client.inference.get_chat_completions_client() as chat_client:
        response = chat_client.complete(
            model=phi4_deployment,
            messages=[system_msg, user_msg],
            temperature=0.8,  # a bit creative
            top_p=0.9,
            max_tokens=400,
        )

    return response.choices[0].message.content

# Example usage:
question = "I'm training for a 5K. Any tips on a weekly workout schedule?"
answer = chat_with_phi4(question, chain_of_thought=True)
print("🗣️ User:", question)
print("🤖 Phi-4:", answer)

🗣️ User: I'm training for a 5K. Any tips on a weekly workout schedule?
🤖 Phi-4: Training for a 5K is a great goal, and having a structured weekly workout schedule can help you prepare effectively. I'll provide a general plan that incorporates a mix of running, cross-training, and rest days. Remember, it's important to tailor any plan to your current fitness level and consult a healthcare professional before starting any new exercise regimen, especially if you have any medical concerns.

### General 5K Training Schedule

**Week 1-4: Building a Base**
1. **Monday:** Rest or Active Recovery
   - Gentle activities like walking or yoga.
   
2. **Tuesday:** Easy Run
   - Start with 20-30 minutes at a comfortable pace.
   
3. **Wednesday:** Cross-Training
   - Engage in activities like cycling, swimming, or strength training for 30-45 minutes.
   
4. **Thursday:** Interval Training
   - Warm up for 5 minutes, then run at a faster pace for 1 minute followed by 2 minutes of walking or jogging. 

## 3. RAG-like Example (Stub)
Phi-4 also excels in retrieval augmented generation scenarios, where you provide external context and let the model reason over it. Below is a **stub** example showing how you'd pass retrieved text as context.

> In a real scenario, you'd embed & search for relevant passages, then feed them into the user/system message.


In [3]:
def chat_with_phi4_rag(user_question, retrieved_doc):
    """Simulate an RAG flow by appending retrieved context to the system prompt."""
    system_prompt = (
        "You are Phi-4, helpful fitness AI.\n"
        "We have some context from the user's knowledge base: \n"
        f"{retrieved_doc}\n"
        "Please use this context to help your answer. If the context doesn't help, say so.\n"
    )

    system_msg = SystemMessage(content=system_prompt)
    user_msg = UserMessage(content=user_question)

    with project_client.inference.get_chat_completions_client() as chat_client:
        response = chat_client.complete(
            model=phi4_deployment,
            messages=[system_msg, user_msg],
            temperature=0.3,
            max_tokens=300,
        )
    return response.choices[0].message.content

# Let's define a dummy doc snippet:
doc_snippet = "Recommended to run 3 times per week and mix with cross-training.\n" \
              "Include rest days or active recovery days for muscle repair."

user_q = "How often should I run weekly to prepare for a 5K?"
rag_answer = chat_with_phi4_rag(user_q, doc_snippet)
print("🗣️ User:", user_q)
print("🤖 Phi-4 (RAG):", rag_answer)

🗣️ User: How often should I run weekly to prepare for a 5K?
🤖 Phi-4 (RAG): To prepare for a 5K, it's great that you're considering a structured running plan. Based on the context you've provided, here's a suggested approach:

1. **Running Frequency**: Aim to run 3 times per week. This frequency allows for consistent training without overloading your body, which is crucial for building endurance and speed.

2. **Cross-Training**: Incorporate cross-training activities on one or two of your non-running days. This can include cycling, swimming, or strength training, which will help improve overall fitness and reduce the risk of injury by balancing muscle groups.

3. **Rest and Recovery**: Include at least one full rest day or an active recovery day each week. Active recovery could involve light activities like walking, yoga, or stretching, which aid in muscle repair and prevent burnout.

4. **Progressive Training**: Gradually increase your running distance and intensity over time. Start wi

## 4. Wrap-Up & Best Practices
1. **Chain-of-Thought**: Great for debugging or certain QA tasks, but be mindful about revealing chain-of-thought to end users.
2. **RAG**: Use `azure-ai-inference` with retrieval results to ground your answers.
3. **OpenTelemetry**: Optionally integrate `opentelemetry-sdk` and `azure-core-tracing-opentelemetry` for full observability.
4. **Evaluate**: Use `azure-ai-evaluation` to measure your model’s performance.
5. **Cost & Performance**: Phi-4 aims to provide near GPT-4 performance at lower cost. Evaluate for your domain needs.

## 🎉 Congratulations!
You've seen how to:
- Use **Phi-4** with `AIProjectClient` and `azure-ai-inference`.
- Create a **chat** flow with chain-of-thought.
- Stub a **RAG** scenario.

> Happy hacking! 🏋️
